In [4]:
from transformers import (
    AutoModelForQuestionAnswering, AutoTokenizer
)

model = AutoModelForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")
tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

In [6]:
tokenizer.decode(tokenizer("A question", "An answer")["input_ids"])

'<s> A question</s></s> An answer</s>'

In [1]:
from dataset import QA
ds = QA()

In [2]:
examples = {
    k: [
        ds[i][k]
        for i in range(10)
    ]
    for k in ds[0].keys()
}

In [3]:
examples

{'document': ['Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответств

In [26]:
def prepare_train_features(
    examples,
    tokenizer,
    max_length,
    doc_stride,
):
    pad_on_right = tokenizer.padding_side == "right"
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["document" if pad_on_right else "document"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        answer_start = examples["answer_start"][sample_index]
        answer_end = examples["answer_end"][sample_index]

        # If no answers are given, set the cls_index as answer.
        if answer_start == answer_end == 0:
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answer_start
            end_char = answer_end

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [18]:
examples.keys()

dict_keys(['document', 'question', 'answer', 'answer_start', 'answer_end'])

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

In [28]:
prepd = prepare_train_features(
    examples, 
    tokenizer=tokenizer, 
    max_length=300, 
    doc_stride=100
)

In [31]:
prepd["input_ids"]

[[0,
  112203,
  133744,
  58514,
  59,
  2,
  2,
  3743,
  2743,
  59585,
  407,
  131658,
  144724,
  695,
  78716,
  49,
  141449,
  53224,
  518,
  40378,
  751,
  438,
  9513,
  3882,
  36605,
  6460,
  12012,
  13821,
  4836,
  910,
  204119,
  245,
  12185,
  171455,
  424,
  2743,
  52217,
  5098,
  3882,
  36605,
  6460,
  12012,
  13821,
  4836,
  910,
  672,
  153009,
  2715,
  97916,
  40378,
  751,
  901,
  84609,
  105168,
  124623,
  221049,
  135705,
  230867,
  15,
  217282,
  59,
  4,
  90216,
  24339,
  16,
  3858,
  179229,
  312,
  15206,
  49,
  2917,
  2716,
  155491,
  49,
  199826,
  158108,
  18102,
  49,
  22326,
  66917,
  173604,
  29,
  19149,
  49,
  40378,
  1269,
  4,
  252,
  5058,
  14496,
  24397,
  1993,
  1089,
  225697,
  2715,
  173604,
  29,
  19149,
  49,
  40378,
  1269,
  3150,
  35231,
  69807,
  989,
  70786,
  419,
  40378,
  751,
  49,
  22692,
  158108,
  18102,
  637,
  71761,
  312,
  24397,
  1993,
  4,
  62739,
  1857,
  6716,
  5,
 

In [32]:
from transformers import default_data_collator

In [35]:
prepd

{'input_ids': [[0, 112203, 133744, 58514, 59, 2, 2, 3743, 2743, 59585, 407, 131658, 144724, 695, 78716, 49, 141449, 53224, 518, 40378, 751, 438, 9513, 3882, 36605, 6460, 12012, 13821, 4836, 910, 204119, 245, 12185, 171455, 424, 2743, 52217, 5098, 3882, 36605, 6460, 12012, 13821, 4836, 910, 672, 153009, 2715, 97916, 40378, 751, 901, 84609, 105168, 124623, 221049, 135705, 230867, 15, 217282, 59, 4, 90216, 24339, 16, 3858, 179229, 312, 15206, 49, 2917, 2716, 155491, 49, 199826, 158108, 18102, 49, 22326, 66917, 173604, 29, 19149, 49, 40378, 1269, 4, 252, 5058, 14496, 24397, 1993, 1089, 225697, 2715, 173604, 29, 19149, 49, 40378, 1269, 3150, 35231, 69807, 989, 70786, 419, 40378, 751, 49, 22692, 158108, 18102, 637, 71761, 312, 24397, 1993, 4, 62739, 1857, 6716, 5, 2678, 128810, 22996, 438, 5896, 9, 81518, 5, 3499, 10698, 15319, 59, 66917, 87819, 70786, 419, 40378, 751, 69378, 5, 170254, 20850, 71761, 312, 24397, 1993, 15856, 529, 69807, 77, 22947, 57443, 135, 69, 1584, 160931, 89993, 195455,

In [56]:
len(prepd["start_positions"]), len(prepd["end_positions"]), len(prepd["input_ids"])

(30, 30, 30)

In [1]:
from dataset import QA
ds = QA()

In [2]:
len(ds)

1798

In [3]:
from transformers import default_data_collator
from torch.utils.data import DataLoader
from datasets import Dataset

def ds_gen():
    for i in range(len(ds)):
        yield ds[i]

ds = Dataset.from_generator(ds_gen)

2023-04-14 15:14:03.994260: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /home/leo/.cache/huggingface/datasets/generator/default-45f161ec5b503e24/0.0.0. Subsequent calls will reuse this data.


In [4]:
from train_utils import prepare_train_features
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [5]:
tokenized_ds = ds.map(
    prepare_train_features, 
    batched=True, 
    remove_columns=ds.column_names,
    fn_kwargs={"tokenizer": tokenizer, "max_length": 400, "doc_stride": 100}
)

Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

In [6]:
dl = DataLoader(tokenized_ds, collate_fn=default_data_collator, batch_size=8)
n = next(iter(dl))


In [7]:
n["input_ids"]

tensor([[  101, 22924, 21168,  ...,   132, 55825,   102],
        [  101, 22924, 21168,  ...,     0,     0,     0],
        [  101, 22924, 21168,  ..., 34108, 41298,   102],
        ...,
        [  101, 22924, 21168,  ...,     0,     0,     0],
        [  101, 22924, 21168,  ...,   132, 19470,   102],
        [  101, 22924, 21168,  ...,     0,     0,     0]])

In [20]:
tokenizer.convert_ids_to_tokens(input_ids[start_position:end_position+1])

['Размер',
 'обеспечения',
 'исполнения',
 'контракта',
 '659',
 '##3',
 '.',
 '25',
 'Российский',
 'рубль']

In [17]:
ds[0]

{'document': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответстви

In [23]:
from spacy import displacy
def ner_render(tokens, start_position, end_position, tag_list=None):
    start_pos = 0
    end_pos = 0

    for i in range(start_position):
        start_pos += len(tokens[i]) + 1 # word + space
    for i in range(end_position+1):
        end_pos += len(tokens[i]) + 1
    ents = [{
        "start": start_pos,
        "end": end_pos,
        "label": ""
    }]
    displacy.render({
        "text": " ".join(tokens),
        "ents": ents,
    }, style="ent", manual=True, jupyter=True)


None


In [27]:
for i in range(10):
    input_ids = n["input_ids"][i]
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    input_string = " ".join(input_tokens)
    start_position = n["start_positions"][i]
    end_position = n["end_positions"][i]
    (ner_render(input_tokens, start_position, end_position))

IndexError: index 8 is out of bounds for dimension 0 with size 8

In [ ]:
import evaluate
evaluate.load("squad")